In [32]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [33]:
#Einlesen
url = "https://www.berufsberatung.ch/dyn/show/2930"
r = requests.get(url)
r = BeautifulSoup(r.text, "html.parser")

In [34]:
#Job IDs finden
jobs = []
for elem in r.find_all('div', {'class', 'row'})[8].find_all('a')[3:]:
    try:
        mini_dict = {"Jobtitel":elem["data-value"], "JobID": elem["data-value2"]}
        jobs.append(mini_dict)
    except:
        continue

In [35]:
df = pd.DataFrame(jobs)

In [78]:
df.head()

,Jobtitel,JobID
0,Abdichter/in EFZ,51914.1
1,Abdichtungspraktiker/in EBA,51919.1
2,Agrarpraktiker/in EBA,15008.2
3,Agrarpraktiker/in (Landwirtschaft) EBA,15009.2
4,Agrarpraktiker/in (Spezialkulturen) EBA,15010.2


In [45]:
#Mit den Lehrstellen IDs alle Lehrstellen nach Berufen auslesen
url = "https://www.berufsberatung.ch/dyn/show/2930?lang=de&Idx=0&OrderBy=1&Order=0&PostBackOrder=0&postBack=true&CountResult=12&Total_Idx=11&CounterSearch=3&UrlAjaxWebSearch=%2FLenaWeb%2FAjaxWebSearch&prof_=51914.1&LocName=&LocId=&Area=10&schoolyear_=2021&schoolyear_=2022&langcode_=de&langcode_=fr&langcode_=it&langcode_=rm&langcode_=en"

In [46]:
r = requests.get(url)
r = BeautifulSoup(r.text, "html.parser")
list_of_jobs = r.find_all('div', {'class':'border-bottom-0-important'})

In [47]:
#Testen für Firmen
r.find_all('div', {'class':'border-bottom-0-important'})[0].find('div', {'class':'table-col-1'}).text

'Jakob Tanner AG'

In [48]:
#Testen für Lehrstelle
r.find_all('div', {'class':'border-bottom-0-important'})[0].find('div', {'class':'table-col-2'}).text

'Abdichter/in EFZ'

In [49]:
#Testen für Orte
r.find_all('div', {'class':'border-bottom-0-important'})[0].find('div', {'class':'table-col-3'}).text

'Aadorf (TG)'

In [50]:
#Testen für Anzahl Lehrstellen
r.find_all('div', {'class':'border-bottom-0-important'})[0].find('div', {'class':'table-col-5'}).text

'\n1 Lehrstelle(n)\n'

In [51]:
#Zusammenführen
finallist = []

for job in list_of_jobs:
    #print(job.find('div', {'class':'table-col-1'}).text)
    company = job.find('div', {'class':'table-col-1'}).text #company
    j = job.find('div', {'class':'table-col-2'}).text #job
    location = job.find('div', {'class':'table-col-3'}).text #loation

    try:
        count = job.find('div', {'class':'table-col-5'}).text #count
    except:
        count = "N/A"
    minidict = {'company': company,
                'job': j,
                'location': location,
                'count':count}
    finallist.append(minidict)


In [52]:
#DataFrame bauen
pd.DataFrame(finallist)

,company,job,location,count
0,Jakob Tanner AG,Abdichter/in EFZ,Aadorf (TG),\n1 Lehrstelle(n)\n
1,Bautherm Flachdach Spenglerei AG,Abdichter/in EFZ,Aarburg (AG),\n1 Lehrstelle(n)\n
2,Abdichtungsbau Durrer AG,Abdichter/in EFZ,Alpnach Dorf (OW),\n2 Lehrstelle(n)\n
3,Abdichtungsbau Durrer AG,Abdichter/in EFZ,Alpnach Dorf (OW),\n2 Lehrstelle(n)\n
4,Carl Meier Sohn AG,Abdichter/in EFZ,Bassersdorf (ZH),\n1 Lehrstelle(n)\n
...,...,...,...,...
85,Maillard Bedachungen AG,Abdichter/in EFZ,Winterthur (ZH),\n1 Lehrstelle(n)\n
86,Böhi + Wirz AG,Abdichter/in EFZ,Ziefen (BL),\n1 Lehrstelle(n)\n
87,Bär Dachtechnik AG,Abdichter/in EFZ,Zofingen (AG),\n1 Lehrstelle(n)\n
88,H. Kreiner AG,Abdichter/in EFZ,Zürich (ZH),\n1 Lehrstelle(n)\n


In [53]:
## Alle Lehrstellen aller Firmen
df['JobID']

0      51914.1
1      51919.1
2      15008.2
3      15009.2
4      15010.2
        ...   
633    64011.1
634    64012.1
635    30303.1
636    54218.1
637           
Name: JobID, Length: 638, dtype: object

In [55]:
all_jobs2 = []

#Mit den Lehrstellen IDs alle Lehrstellen nach Berufen auslesen
url1 = "https://www.berufsberatung.ch/dyn/show/2930?lang=de&Idx=0&OrderBy=1&Order=0&PostBackOrder=0&postBack=true&CountResult=1000&Total_Idx=11&CounterSearch=3&UrlAjaxWebSearch=%2FLenaWeb%2FAjaxWebSearch&prof_="
url2 = "&LocName=&LocId=&Area=10&schoolyear_=2021&schoolyear_=2022&langcode_=de&langcode_=fr&langcode_=it&langcode_=rm&langcode_=en"

for ID in df['JobID']:
    r = requests.get(url1+ID+url2)
    r = BeautifulSoup(r.text, "html.parser")
    list_of_jobs = r.find_all('div', {'class':'border-bottom-0-important'})
    
    finallist = []

    for job in list_of_jobs:
        #print(job.find('div', {'class':'table-col-1'}).text)
        company = job.find('div', {'class':'table-col-1'}).text #company
        j = job.find('div', {'class':'table-col-2'}).text #job
        location = job.find('div', {'class':'table-col-3'}).text #loation

        try:
            count = job.find('div', {'class':'table-col-5'}).text #count
        except:
            count = "N/A"
        minidict = {'company': company,
                'job': j,
                'location': location,
                'count':count}
        finallist.append(minidict)
    
    all_jobs = all_jobs + finallist
    print(ID)

51914.1
51919.1
15008.2
15009.2
15010.2
15011.2
44727.1
44702.1
40105.1
54603.1
86913.1
85506.1
47416.2
46426.2
46318.1
46324.1
46326.1
46325.1
46321.1
46323.1
46322.1
21107.1
21104.2
21105.2
21106.2
43906.1
51007.1
51005.1
27121.1
27121.6
27121.2
27121.3
27121.5
27121.4
27122.1
39905.1
88614.1
54215.1
54215.2
54215.3
51706.2
51708.2
51707.2
30405.1
30404.1
70512.1
43102.1
90904.1
90905.1
90907.1
90906.1
68103.2
45307.1
45308.1
45306.1
37006.1
37005.1
37005.2
37005.3
37005.4
82013.1
82014.1
51915.1
51920.1
28503.1
86916.1
71800.1
71801.1
71802.1
71803.1
71804.1
71805.1
71806.1
71807.1
71808.1
71809.1
71810.1
71811.1
71812.1
71813.1
71814.1
71815.1
71816.1
71817.1
71818.1
71819.1
71820.1
71821.1
71822.1
71400.1
71495.1
71412.1
71405.1
71429.1
71426.1
71425.1
71423.1
71414.1
71493.1
71420.1
71427.1
71497.1
71494.1
71418.1
71424.1
71492.1
71411.1
71430.1
71422.1
71419.1
71498.1
71417.1
71409.1
71410.1
71428.1
71416.1
71406.1
71421.1
71900.3
71901.3
71902.3
71903.3
71904.3
71905.3
71906.3


In [54]:
df['JobID']

0      51914.1
1      51919.1
2      15008.2
3      15009.2
4      15010.2
        ...   
633    64011.1
634    64012.1
635    30303.1
636    54218.1
637           
Name: JobID, Length: 638, dtype: object

In [56]:
len(all_jobs)

25692

In [58]:
len(all_jobs)

25692

In [59]:
pd.DataFrame(all_jobs).to_csv("lehren2.csv")

In [60]:
df2 = pd.DataFrame(all_jobs)

In [65]:
list(df2)

['company', 'job', 'location', 'count']

In [68]:
def clean(elem):
    elem = elem.replace("\n", "")
    elem = elem.split(" ")[0]
    elem = int(elem)
    return elem
df2['count'] = df2['count'].apply(clean)

In [69]:
df2['count'].sum()

33727

In [72]:
df2.groupby('company')['count'].sum().sort_values()

company
1001 Carreaux                                           1
Physio in Fit GmbH                                      1
Physio Training Wilchingen                              1
Photo Vision Zumstein SA                                1
Photo Vision SA                                         1
                                                     ... 
Coop Genossenschaft                                   197
Volg Detailhandels AG                                 210
Zürcher Kantonalbank                                  227
Places d'apprentissage dans tout le canton de Vaud    326
Post CH AG                                            435
Name: count, Length: 8890, dtype: int64